- regnet160
- patience=50
- epochs=500
- crop 추가
- batch_size=16

- Ubuntu 18.04, Cuda 11
- opencv-python
- numpy
- pandas
- timm
- torch==1.8.0 torchvision 0.9.0 with cuda 11.1
- natsort
- scikit-learn
- pillow
- torch_optimizer
- tqdm
- ptflops
- easydict
- matplotlib

# Library

In [2]:
import os
import cv2
import time
import random
import logging  # 로그 출력
import easydict  # 속성으로 dict 값에 access할 수 있음
import numpy as np
import pandas as pd
from tqdm import tqdm  # process bar
from os.path import join as opj
from ptflops import get_model_complexity_info
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
# from PIL import Image

import timm
import torch
import torch.nn as nn
import torch_optimizer as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, grad_scaler
from torchvision import transforms

import warnings
warnings.filterwarnings('ignore')

# Config

Hyper-parameter 정의

In [3]:
args = easydict.EasyDict(
    {'exp_num':'0',
     
     # Path settings
     'data_path':'./data',
     'Kfold':5,
     'model_path':'results/',

     # Model parameter settings  # 고려사항
#      'encoder_name':'regnety_040',  
     'encoder_name':'regnety_160',
#      'encoder_name':'efficientnet_b3',
     'drop_path_rate':0.2,
     
     # Training parameter settings
     ## Base Parameter
     'img_size':224,  # 고려사항
     'batch_size':16, # 고려사항
#      'epochs':100,  # 고려사항
     'epochs':500,
     'optimizer':'Lamb',
     'initial_lr':5e-6,
     'weight_decay':1e-3,

     ## Augmentation
     'aug_ver':2,

     ## Scheduler (OnecycleLR)
     'scheduler':'cycle',
     'warm_epoch':5,
     'max_lr':1e-3,

     ### Cosine Annealing
     'min_lr':5e-6,
     'tmax':145,

     ## etc.
     'patience':50,
     'clipping':None,

     # Hardware settings
     'amp':True,
     'multi_gpu':False,
     'logging':False,
     'num_workers':0,  # RuntimeError: DataLoader worker 오류 발생으로 0으로 설정
#      'num_workers':4,  # 고려사항
     'seed':42
    })

# Utils for training and Logging

In [4]:
# Warmup Learning rate scheduler
from torch.optim.lr_scheduler import _LRScheduler
class WarmUpLR(_LRScheduler):
    """warmup_training learning rate scheduler
    Args:
        optimizer: optimizer(e.g. SGD)
        total_iters: totoal_iters of warmup phase
    """
    def __init__(self, optimizer, total_iters, last_epoch=-1):
        
        self.total_iters = total_iters
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        """we will use the first m batches, and set the learning
        rate to base_lr * m / total_iters
        """
        return [base_lr * self.last_epoch / (self.total_iters + 1e-8) for base_lr in self.base_lrs]

# Logging
def get_root_logger(logger_name='basicsr',
                    log_level=logging.INFO,
                    log_file=None):

    logger = logging.getLogger(logger_name)
    # if the logger has been initialized, just return it
    if logger.hasHandlers():
        return logger

    format_str = '%(asctime)s %(levelname)s: %(message)s'
    logging.basicConfig(format=format_str, level=log_level)

    if log_file is not None:
        file_handler = logging.FileHandler(log_file, 'w')
        file_handler.setFormatter(logging.Formatter(format_str))
        file_handler.setLevel(log_level)
        logger.addHandler(file_handler)

    return logger

class AvgMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        self.losses = []

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        self.losses.append(val)

# Data Preprocessing
- 원본 이미지 사이즈가 큰 점을 감안해 (256,256)로 resize하여 데이터를 새롭게 저장

In [4]:
# df = pd.read_csv('./data/train_df.csv')

# # Resize Train Images
# save_path = './data/train_256_new'  # 새로 저장할 폴더 경로
# os.makedirs(save_path, exist_ok=True)
# for img in tqdm(df['file_name']):  # train_df의 'file_name' 컬럼을 참고하여
#     name = os.path.basename(img)
#     img = cv2.imread(opj('./data/train/', img))  # 해당 경로에 있는 png 이미지 읽어서
# #     img = cv2.resize(img, dsize=(256, 256))  # resize한 후
#     img = cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_AREA) # 고려사항
#     img = cv2.imwrite(opj(save_path, name), img)  # 새 폴더에 저장

# # Resize Test Images
# df = pd.read_csv('./data/test_df.csv')
# save_path = './data/test_256_new'
# os.makedirs(save_path, exist_ok=True)
# for img in tqdm(df['file_name']):
#     name = os.path.basename(img)
#     img = cv2.imread(opj('./data/test/', img))
# #     img = cv2.resize(img, dsize=(256, 256))
#     img = cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_AREA) # 고려사항
#     img = cv2.imwrite(opj(save_path, name), img)

# Dataset & Loader

In [5]:
class Train_Dataset(Dataset):
    def __init__(self, df, transform=None):
        self.file_name = df['file_name'].values
        # 각 label을 str->index로 변환
        labels = ['bottle-broken_large', 'bottle-broken_small', 'bottle-contamination', 'bottle-good', 'cable-bent_wire', 'cable-cable_swap', 'cable-combined', 'cable-cut_inner_insulation', 'cable-cut_outer_insulation', 'cable-good', 'cable-missing_cable', 'cable-missing_wire', 'cable-poke_insulation', 'capsule-crack', 'capsule-faulty_imprint', 'capsule-good', 'capsule-poke', 'capsule-scratch', 'capsule-squeeze', 'carpet-color', 'carpet-cut', 'carpet-good', 'carpet-hole', 'carpet-metal_contamination', 'carpet-thread', 'grid-bent', 'grid-broken', 'grid-glue', 'grid-good', 'grid-metal_contamination', 'grid-thread', 'hazelnut-crack', 'hazelnut-cut', 'hazelnut-good', 'hazelnut-hole', 'hazelnut-print', 'leather-color', 'leather-cut', 'leather-fold', 'leather-glue', 'leather-good', 'leather-poke', 'metal_nut-bent', 'metal_nut-color', 'metal_nut-flip', 'metal_nut-good', 'metal_nut-scratch', 'pill-color', 'pill-combined', 'pill-contamination', 'pill-crack', 'pill-faulty_imprint', 'pill-good', 'pill-pill_type', 'pill-scratch', 'screw-good', 'screw-manipulated_front', 'screw-scratch_head', 'screw-scratch_neck', 'screw-thread_side', 'screw-thread_top', 'tile-crack', 'tile-glue_strip', 'tile-good', 'tile-gray_stroke', 'tile-oil', 'tile-rough', 'toothbrush-defective', 'toothbrush-good', 'transistor-bent_lead', 'transistor-cut_lead', 'transistor-damaged_case', 'transistor-good', 'transistor-misplaced', 'wood-color', 'wood-combined', 'wood-good', 'wood-hole', 'wood-liquid', 'wood-scratch', 'zipper-broken_teeth', 'zipper-combined', 'zipper-fabric_border', 'zipper-fabric_interior', 'zipper-good', 'zipper-rough', 'zipper-split_teeth', 'zipper-squeezed_teeth']
        new = dict(zip(range(len(labels)),labels))
        label_decoder = {val:key for key, val in new.items()}
        df['label'] = df['label'].replace(label_decoder)

        self.target = df['label'].values  # 목표는 label
        self.transform = transform

        print(f'Dataset size:{len(self.file_name)}')

    def __getitem__(self, idx):  # train 경로에 있는 png 이미지 읽어서 float32로 변환
        image = cv2.imread(opj('./data/train_256_new/', self.file_name[idx])).astype(np.float32)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0  # BGR=>RGB 변환
        
        # PIL 사용 # 고려사항
#         image = Image.open(opj('./data/train_256/', self.file_name[idx])).convert('RGB')
#         image = self.transform(image)
        
        target = self.target[idx]
#         print(f'target:{target}')

        if self.transform is not None:
        # HWC => CHW-layout 변환
            image = self.transform(torch.from_numpy(image.transpose(2,0,1)))

        return image, target

    def __len__(self):
        return len(self.file_name)

class Test_dataset(Dataset):
    def __init__(self, df, transform=None):
        self.test_file_name = df['file_name'].values
        self.transform = transform

        print(f'Test Dataset size:{len(self.test_file_name)}')

    def __getitem__(self, idx): # test 경로에 있는 png 이미지 읽어서 float32로 변환
        image = cv2.imread(opj('./data/test_256_new/', self.test_file_name[idx])).astype(np.float32)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0  # BGR=>RGB 변환

        if self.transform is not None:
            image = self.transform(torch.from_numpy(image.transpose(2,0,1)))

        return image

    def __len__(self):
        return len(self.test_file_name)

def get_loader(df, phase: str, batch_size, shuffle,
               num_workers, transform):
    if phase == 'test':
        dataset = Test_dataset(df, transform)  
        # num_workers : 데이터 로딩에 사용하는 subprocess 개수
        # pin_memory : True - 데이터로더가 Tensor를 CUDA 고정 메모리에 올림
        # drop_last : batch의 크기에 따른 의존도 높은 함수를 사용할 때 우려되는 경우 마지막 batch를 사용하지 않을 수 있음
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True)
    else:
        dataset = Train_Dataset(df, transform)
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True,
                                 drop_last=False)
    return data_loader

def get_train_augmentation(img_size, ver):
    if ver == 1: # for validset
        transform = transforms.Compose([
#                 transforms.Resize((img_size, img_size)),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                ])

    if ver == 2:
        transform = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),  # 추가
                transforms.RandomCrop(224),  # 추가
                transforms.RandomPerspective(),  # 추가
                transforms.RandomAffine((20)),  # x, y축으로 이미지 늘림
                transforms.RandomRotation(90),
#                 transforms.Resize((img_size, img_size)),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
            ])
    
    
    return transform

# Network

In [6]:
class Network(nn.Module):
    def __init__(self, args):
        super().__init__()
        # 사전 학습된 모델 사용하기
        self.encoder = timm.create_model(args.encoder_name, pretrained=True,
                                    drop_path_rate=args.drop_path_rate,
                                    )
#         num_head = self.encoder.head.fc.in_features  # Number of parallel attention heads
#         self.encoder.head.fc = nn.Linear(num_head, 88)
        
        if 'regnet' in args.encoder_name:        
            num_head = self.encoder.head.fc.in_features
            self.encoder.head.fc = nn.Linear(num_head, 88)
        
        elif 'efficient' in args.encoder_name:
            num_head = self.encoder.classifier.in_features
            self.encoder.classifier = nn.Linear(num_head, 88)

    def forward(self, x):
        x = self.encoder(x)
        return x

class Network_test(nn.Module):
    def __init__(self, encoder_name):
        super().__init__()
        self.encoder = timm.create_model(encoder_name, pretrained=True,
                                    drop_path_rate=0,
                                    )
#         num_head = self.encoder.head.fc.in_features
#         self.encoder.head.fc = nn.Linear(num_head, 88)
        
        if 'regnet' in encoder_name:        
            num_head = self.encoder.head.fc.in_features
            self.encoder.head.fc = nn.Linear(num_head, 88)
        
        elif 'efficient' in encoder_name:
            num_head = self.encoder.classifier.in_features
            self.encoder.classifier = nn.Linear(num_head, 88)
    
    def forward(self, x):
        x = self.encoder(x)
        return x

# Trainer for Training & Validation

In [7]:
class Trainer():
    def __init__(self, args, save_path):
        '''
        args: arguments
        save_path: Model 가중치 저장 경로
        '''
        super(Trainer, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Logging
        log_file = os.path.join(save_path, 'log_0512_2.log')
        self.logger = get_root_logger(logger_name='IR', log_level=logging.INFO, log_file=log_file)
        self.logger.info(args)
        # self.logger.info(args.tag)

        # Train, Valid Set load
        ############################################################################
        df_train = pd.read_csv(opj(args.data_path, 'train_df.csv'))
        print('Read train_df.csv')
            
#         if args.step == 0 :
#             df_train = pd.read_csv(opj(args.data_path, 'train_df.csv'))
#             print('Read train_df.csv')
#         else :
#             df_train = pd.read_csv(opj(args.data_path, f'aws_0505_esb_train_{args.step}step.csv'))
#             print(f'Read {args.step}step.csv')

#         if args.image_type is not None:
#             df_train['file_name'] = df_train['file_name'].apply(lambda x:x.replace('train_imgs', args.image_type))
#             df_train['file_name'] = df_train['file_name'].apply(lambda x:x.replace('test_imgs', 'test_512'))

        kf = StratifiedKFold(n_splits=args.Kfold, shuffle=True, random_state=args.seed)
        for fold, (train_idx, val_idx) in enumerate(kf.split(range(len(df_train)), y=df_train['label'])):
            df_train.loc[val_idx, 'fold'] = fold
        val_idx = list(df_train[df_train['fold'] == int(args.fold)].index)

        df_val = df_train[df_train['fold'] == args.fold].reset_index(drop=True)
        df_train = df_train[df_train['fold'] != args.fold].reset_index(drop=True)

        # Augmentation
        self.train_transform = get_train_augmentation(img_size=args.img_size, ver=args.aug_ver)
        self.test_transform = get_train_augmentation(img_size=args.img_size, ver=1)

        # TrainLoader
        self.train_loader = get_loader(df_train, phase='train', batch_size=args.batch_size, shuffle=True,
                                       num_workers=args.num_workers, transform=self.train_transform)
        self.val_loader = get_loader(df_val, phase='train', batch_size=args.batch_size, shuffle=False,
                                       num_workers=args.num_workers, transform=self.test_transform)

        # Network
        self.model = Network(args).to(self.device)
        macs, params = get_model_complexity_info(self.model, (3, args.img_size, args.img_size), as_strings=True,
                                                 print_per_layer_stat=False, verbose=False)
        self.logger.info('{:<30}  {:<8}'.format('Computational complexity: ', macs))
        self.logger.info('{:<30}  {:<8}'.format('Number of parameters: ', params))

        # Loss
        self.criterion = nn.CrossEntropyLoss()
        
        # Optimizer & Scheduler
        self.optimizer = optim.Lamb(self.model.parameters(), lr=args.initial_lr, weight_decay=args.weight_decay)
        
        iter_per_epoch = len(self.train_loader)
        self.warmup_scheduler = WarmUpLR(self.optimizer, iter_per_epoch * args.warm_epoch)

        if args.scheduler == 'step':
            self.scheduler = torch.optim.lr_scheduler.MultiStepLR(self.optimizer, milestones=args.milestone, gamma=args.lr_factor, verbose=True)
        elif args.scheduler == 'cos':
            tmax = args.tmax # half-cycle 
            self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max = tmax, eta_min=args.min_lr, verbose=True)
        elif args.scheduler == 'cycle':
            self.scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer, max_lr=args.max_lr, steps_per_epoch=iter_per_epoch, epochs=args.epochs)

        if args.multi_gpu:
            self.model = nn.DataParallel(self.model).to(self.device)

        # Train / Validate
        best_loss = np.inf
        best_acc = 0
        best_epoch = 0
        early_stopping = 0
        start = time.time()
        
        # 추가
        new_batch_size = args.batch_size
        cnt = 0
        
        for epoch in range(1, args.epochs+1):
            self.epoch = epoch

            if args.scheduler == 'cos':
                if epoch > args.warm_epoch:
                    self.scheduler.step()

            # Training
            train_loss, train_acc, train_f1 = self.training(args)

            # Model weight in Multi_GPU or Single GPU
            state_dict= self.model.module.state_dict() if args.multi_gpu else self.model.state_dict()

            # Validation
            val_loss, val_acc, val_f1 = self.validate(args, phase='val')

            # Save models
            if val_loss < best_loss:
                early_stopping = 0
                best_epoch = epoch
                best_loss = val_loss
                best_acc = val_acc
                best_f1 = val_f1

                torch.save({'epoch':epoch,
                            'state_dict':state_dict,
                            'optimizer': self.optimizer.state_dict(),
                            'scheduler': self.scheduler.state_dict(),
                    }, os.path.join(save_path, 'best_model_0512_2.pth'))
                self.logger.info(f'-----------------SAVE:{best_epoch}epoch----------------')
            else:
                early_stopping += 1

            # Early Stopping
            if early_stopping == args.patience:
                
                cnt = 0
                break
                
            # 추가 - val loss 향상되지 않으면 batch size 조정
            
#             if early_stopping >= 5 * (cnt + 1):
#                 new_batch_size *= 2
#                 cnt += 1
#                 # TrainLoader
#                 self.train_loader = get_loader(df_train, phase='train', batch_size=new_batch_size, shuffle=True,
#                                                num_workers=args.num_workers, transform=self.train_transform)
#                 self.val_loader = get_loader(df_val, phase='train', batch_size=new_batch_size, shuffle=False,
#                                                num_workers=args.num_workers, transform=self.test_transform)
#                 print(f'\nchange batch size to {new_batch_size}')
                
            print(f'\nbest epoch:{best_epoch}/loss:{best_loss:.4f}/f1:{best_f1:.4f}')

        self.logger.info(f'\nBest Val Epoch:{best_epoch} | Val Loss:{best_loss:.4f} | Val Acc:{best_acc:.4f} | Val F1:{best_f1:.4f}')
        end = time.time()
        self.logger.info(f'Total Process time:{(end - start) / 60:.3f}Minute')

    # Training
    def training(self, args):
        self.model.train()
        train_loss = AvgMeter()
        train_acc = 0
        preds_list = []
        targets_list = []

        scaler = grad_scaler.GradScaler()
        for i, (images, targets) in enumerate(tqdm(self.train_loader)):
            images = torch.tensor(images, device=self.device, dtype=torch.float32)
            # ValueError: too many dimensions 'str'
#             targets = torch.tensor(int(targets), device=self.device, dtype=torch.long)
            targets = torch.tensor(targets, device=self.device, dtype=torch.long)
            
            if self.epoch <= args.warm_epoch:
                self.warmup_scheduler.step()

            self.model.zero_grad(set_to_none=True)
            if args.amp:
                with autocast():
                    preds = self.model(images)
                    loss = self.criterion(preds, targets)
                scaler.scale(loss).backward()

                # Gradient Clipping
                if args.clipping is not None:
                    scaler.unscale_(self.optimizer)
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), args.clipping)

                scaler.step(self.optimizer)
                scaler.update()

            else:
                preds = self.model(images)
                loss = self.criterion(preds, targets)
                loss.backward()
                nn.utils.clip_grad_norm_(self.model.parameters(), args.clipping)
                self.optimizer.step()

            if args.scheduler == 'cycle':
                if self.epoch > args.warm_epoch:
                    self.scheduler.step()

            # Metric
            train_acc += (preds.argmax(dim=1) == targets).sum().item()
            preds_list.extend(preds.argmax(dim=1).cpu().detach().numpy())
            targets_list.extend(targets.cpu().detach().numpy())
            # log
            train_loss.update(loss.item(), n=images.size(0))

        train_acc /= len(self.train_loader.dataset)
        train_f1 = f1_score(np.array(targets_list), np.array(preds_list), average='macro')

        self.logger.info(f'Epoch:[{self.epoch:03d}/{args.epochs:03d}]')
        self.logger.info(f'Train Loss:{train_loss.avg:.3f} | Acc:{train_acc:.4f} | F1:{train_f1:.4f}')
        return train_loss.avg, train_acc, train_f1
            
    # Validation or Dev
    def validate(self, args, phase='val'):
        self.model.eval()
        with torch.no_grad():
            val_loss = AvgMeter()
            val_acc = 0
            preds_list = []
            targets_list = []

            for i, (images, targets) in enumerate(self.val_loader):
                images = torch.tensor(images, device=self.device, dtype=torch.float32)
                targets = torch.tensor(targets, device=self.device, dtype=torch.long)

                preds = self.model(images)
                loss = self.criterion(preds, targets)

                # Metric
                val_acc += (preds.argmax(dim=1) == targets).sum().item()
                preds_list.extend(preds.argmax(dim=1).cpu().detach().numpy())
                targets_list.extend(targets.cpu().detach().numpy())

                # log
                val_loss.update(loss.item(), n=images.size(0))
            val_acc /= len(self.val_loader.dataset)
            val_f1 = f1_score(np.array(targets_list), np.array(preds_list), average='macro')

            self.logger.info(f'{phase} Loss:{val_loss.avg:.3f} | Acc:{val_acc:.4f} | F1:{val_f1:.4f}')
        return val_loss.avg, val_acc, val_f1

# Main

In [8]:
def main(args):
    print('<---- Training Params ---->')
    
    # Random Seed
    seed = args.seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True

    save_path = os.path.join(args.model_path, (args.exp_num).zfill(3))
    
    # Create model directory
    os.makedirs(save_path, exist_ok=True)
    Trainer(args, save_path)

    return save_path

# Inference & Make pseudo label set

In [9]:
def predict(encoder_name, test_loader, device, model_path):
    model = Network_test(encoder_name).to(device)
    model.load_state_dict(torch.load(opj(model_path, 'best_model_0512_2.pth'))['state_dict'])
    model.eval()
    preds_list = []
    with torch.no_grad():
        for images in tqdm(test_loader):
            images = torch.as_tensor(images, device=device, dtype=torch.float32)
            preds = model(images)
            preds = torch.softmax(preds, dim=1)
            preds_list.extend(preds.cpu().tolist())

    return np.array(preds_list)

def ensemble_5fold(model_path_list, test_loader, device):
    predict_list = []
    for model_path in model_path_list:  # 고려사항
#         prediction = predict(encoder_name= 'efficientnet_b3', 
        prediction = predict(encoder_name= 'regnety_160', 
#         prediction = predict(encoder_name= 'regnety_040', 
                             test_loader = test_loader, device = device, model_path = model_path)
        predict_list.append(prediction)
    ensemble = (predict_list[0] + predict_list[1] + predict_list[2] + predict_list[3] + predict_list[4])/len(predict_list)

    return ensemble


# def make_pseudo_df(train_df, test_df, ensemble, step, threshold = 0.9, z_sample = 500): 
#     train_df_copy = train_df.copy()
#     test_df_copy = test_df.copy()

#     test_df_copy['label'] = ensemble.argmax(axis=1)
#     pseudo_test_df = test_df_copy.iloc[np.where(ensemble > threshold)[0]].reset_index(drop=True)
#     z_idx  = pseudo_test_df[pseudo_test_df['label'] == 0].sample(n=z_sample, random_state=42).index.tolist()
#     ot_idx = pseudo_test_df[pseudo_test_df['label'].isin([*range(1,88)])].index.tolist()  # 고려사항
#     pseudo_test_df = pseudo_test_df.iloc[z_idx + ot_idx]

#     train_df_copy = train_df_copy.append(pseudo_test_df, ignore_index=True).reset_index(drop=True) # reset_index
#     train_df_copy.to_csv(f'./data/0505_1_train_{step}step.csv', index=False)
#     print(f'Make train_{step}step.csv')

# Train & Inference
- 5fold Training -> Inference & Ensemble -> Make or Update Pseudo label set -> Add Dataset(Trainset + Pseudo label set)
다음과 과정을 반복하기 때문에 Training과 Inference를 동시에 진행했습니다.

In [10]:
# img_size = 256  # 고려사항
img_size = 224
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sub = pd.read_csv('./data/sample_submission.csv')
df_train = pd.read_csv('./data/train_df.csv')
df_test = pd.read_csv('./data/test_df.csv')

In [11]:
# df_test['file_name'] = df_test['file_name'].apply(lambda x:x.replace('test_imgs', 'test_1024'))
test_transform = get_train_augmentation(img_size=img_size, ver=1)
test_dataset = Test_dataset(df_test, test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)  # 고려사항

Test Dataset size:2154


In [ ]:
start = 0 # first time : Only Trainset
# steps = 6 # Number of pseudo labeling times 
models_path = []
for s_fold in range(5): # 5fold
    args.fold = s_fold
    args.exp_num = str(s_fold)
    save_path = main(args)
    models_path.append(save_path)
    
# pseudo labeling하는 경우
# for step in range(start, steps+1): 
#     models_path = []
#     args.step = step
#     for s_fold in range(5): # 5fold
#         args.fold = s_fold
#         args.exp_num = str(s_fold)
#         save_path = main(args)
#         models_path.append(save_path)
#     ensemble = ensemble_5fold(models_path, test_loader, device)
#     make_pseudo_df(df_train, df_test, ensemble, step+1)

2022-05-12 09:59:12,041 INFO: {'exp_num': '0', 'data_path': './data', 'Kfold': 5, 'model_path': 'results/', 'encoder_name': 'regnety_160', 'drop_path_rate': 0.2, 'img_size': 224, 'batch_size': 16, 'epochs': 500, 'optimizer': 'Lamb', 'initial_lr': 5e-06, 'weight_decay': 0.001, 'aug_ver': 2, 'scheduler': 'cycle', 'warm_epoch': 5, 'max_lr': 0.001, 'min_lr': 5e-06, 'tmax': 145, 'patience': 50, 'clipping': None, 'amp': True, 'multi_gpu': False, 'logging': False, 'num_workers': 0, 'seed': 42, 'fold': 0}


<---- Training Params ---->
Read train_df.csv
Dataset size:3421
Dataset size:856


2022-05-12 09:59:12,808 INFO: Loading pretrained weights from url (https://dl.fbaipublicfiles.com/deit/regnety_160-a5fe301d.pth)
2022-05-12 09:59:15,966 INFO: Computational complexity:       15.93 GMac
2022-05-12 09:59:15,967 INFO: Number of parameters:           80.83 M 
100%|████████████████████████████████████████████████████████████████████████████████| 214/214 [01:59<00:00,  1.79it/s]
2022-05-12 10:01:15,858 INFO: Epoch:[001/500]
2022-05-12 10:01:15,859 INFO: Train Loss:4.484 | Acc:0.0132 | F1:0.0049
2022-05-12 10:01:27,053 INFO: val Loss:4.537 | Acc:0.0000 | F1:0.0000
2022-05-12 10:01:28,797 INFO: -----------------SAVE:1epoch----------------



best epoch:1/loss:4.5366/f1:0.0000


100%|████████████████████████████████████████████████████████████████████████████████| 214/214 [01:50<00:00,  1.93it/s]
2022-05-12 10:03:19,679 INFO: Epoch:[002/500]
2022-05-12 10:03:19,679 INFO: Train Loss:4.470 | Acc:0.0146 | F1:0.0041
2022-05-12 10:03:29,447 INFO: val Loss:4.527 | Acc:0.0000 | F1:0.0000
2022-05-12 10:03:31,337 INFO: -----------------SAVE:2epoch----------------


...

100%|████████████████████████████████████████████████████████████████████████████████| 214/214 [01:53<00:00,  1.89it/s]
2022-05-13 01:42:55,107 INFO: Epoch:[091/500]
2022-05-13 01:42:55,107 INFO: Train Loss:0.116 | Acc:0.9696 | F1:0.8795
2022-05-13 01:43:05,034 INFO: val Loss:0.178 | Acc:0.9556 | F1:0.7859



best epoch:73/loss:0.1155/f1:0.8226


100%|████████████████████████████████████████████████████████████████████████████████| 214/214 [01:52<00:00,  1.90it/s]
2022-05-13 01:44:57,914 INFO: Epoch:[092/500]
2022-05-13 01:44:57,915 INFO: Train Loss:0.113 | Acc:0.9705 | F1:0.8958
2022-05-13 01:45:07,837 INFO: val Loss:0.149 | Acc:0.9532 | F1:0.7842



best epoch:73/loss:0.1155/f1:0.8226


 72%|█████████████████████████████████████████████████████████▌                      | 154/214 [01:21<00:32,  1.87it/s]

In [ ]:
# def ensemble_fold(model_path_list, test_loader, device):
#     predict_list = []
#     for model_path in model_path_list:  # 고려사항
# #         prediction = predict(encoder_name= 'efficientnet_b3', 
#         prediction = predict(encoder_name= 'regnety_080', 
# #         prediction = predict(encoder_name= 'regnety_040', 
#                              test_loader = test_loader, device = device, model_path = model_path)
#         predict_list.append(prediction)
#     ensemble = predict_list[0]

#     return ensemble

이어서 학습하기 - 0,1 진행 후 2에서 끊김

In [1]:
models_path=[]

In [12]:
save_path = os.path.join(args.model_path, (str(0)).zfill(3))
models_path.append(save_path)

save_path = os.path.join(args.model_path, (str(1)).zfill(3))
models_path.append(save_path)

In [13]:
models_path

['results/000', 'results/001']

In [ ]:
for s_fold in range(2, 5): # 5fold
    args.fold = s_fold
    args.exp_num = str(s_fold)
    save_path = main(args)
    models_path.append(save_path)

2022-05-13 02:02:58,548 INFO: {'exp_num': '2', 'data_path': './data', 'Kfold': 5, 'model_path': 'results/', 'encoder_name': 'regnety_160', 'drop_path_rate': 0.2, 'img_size': 224, 'batch_size': 16, 'epochs': 500, 'optimizer': 'Lamb', 'initial_lr': 5e-06, 'weight_decay': 0.001, 'aug_ver': 2, 'scheduler': 'cycle', 'warm_epoch': 5, 'max_lr': 0.001, 'min_lr': 5e-06, 'tmax': 145, 'patience': 50, 'clipping': None, 'amp': True, 'multi_gpu': False, 'logging': False, 'num_workers': 0, 'seed': 42, 'fold': 2}


<---- Training Params ---->
Read train_df.csv
Dataset size:3422
Dataset size:855


2022-05-13 02:02:59,319 INFO: Loading pretrained weights from url (https://dl.fbaipublicfiles.com/deit/regnety_160-a5fe301d.pth)
2022-05-13 02:03:03,825 INFO: Computational complexity:       15.93 GMac
2022-05-13 02:03:03,825 INFO: Number of parameters:           80.83 M 
100%|████████████████████████████████████████████████████████████████████████████████| 214/214 [02:54<00:00,  1.23it/s]
2022-05-13 02:05:57,896 INFO: Epoch:[001/500]
2022-05-13 02:05:57,896 INFO: Train Loss:4.487 | Acc:0.0099 | F1:0.0052
2022-05-13 02:06:14,877 INFO: val Loss:4.533 | Acc:0.0000 | F1:0.0000
2022-05-13 02:06:16,558 INFO: -----------------SAVE:1epoch----------------



best epoch:1/loss:4.5334/f1:0.0000


100%|████████████████████████████████████████████████████████████████████████████████| 214/214 [01:50<00:00,  1.94it/s]
2022-05-13 02:08:06,990 INFO: Epoch:[002/500]
2022-05-13 02:08:06,991 INFO: Train Loss:4.469 | Acc:0.0134 | F1:0.0045
2022-05-13 02:08:16,690 INFO: val Loss:4.519 | Acc:0.0012 | F1:0.0002
2022-05-13 02:08:18,955 INFO: -----------------SAVE:2epoch----------------


...

100%|████████████████████████████████████████████████████████████████████████████████| 214/214 [01:54<00:00,  1.87it/s]
2022-05-13 10:50:05,938 INFO: Epoch:[131/500]
2022-05-13 10:50:05,939 INFO: Train Loss:0.106 | Acc:0.9728 | F1:0.8985
2022-05-13 10:50:15,865 INFO: val Loss:0.154 | Acc:0.9591 | F1:0.8029



best epoch:118/loss:0.1222/f1:0.8505


100%|████████████████████████████████████████████████████████████████████████████████| 214/214 [01:54<00:00,  1.87it/s]
2022-05-13 10:52:10,461 INFO: Epoch:[132/500]
2022-05-13 10:52:10,462 INFO: Train Loss:0.105 | Acc:0.9684 | F1:0.8793
2022-05-13 10:52:20,374 INFO: val Loss:0.153 | Acc:0.9637 | F1:0.8290



best epoch:118/loss:0.1222/f1:0.8505


 39%|███████████████████████████████▍                                                 | 83/214 [00:44<01:10,  1.86it/s]

### 4 추가 후 합치기

### ensemble 진행

In [ ]:
ensemble = ensemble_5fold(models_path, test_loader, device)

In [ ]:
# For submission
sub.iloc[:, 1] = ensemble.argmax(axis=1)
labels = ['bottle-broken_large', 'bottle-broken_small', 'bottle-contamination', 'bottle-good', 'cable-bent_wire', 'cable-cable_swap', 'cable-combined', 'cable-cut_inner_insulation', 'cable-cut_outer_insulation', 'cable-good', 'cable-missing_cable', 'cable-missing_wire', 'cable-poke_insulation', 'capsule-crack', 'capsule-faulty_imprint', 'capsule-good', 'capsule-poke', 'capsule-scratch', 'capsule-squeeze', 'carpet-color', 'carpet-cut', 'carpet-good', 'carpet-hole', 'carpet-metal_contamination', 'carpet-thread', 'grid-bent', 'grid-broken', 'grid-glue', 'grid-good', 'grid-metal_contamination', 'grid-thread', 'hazelnut-crack', 'hazelnut-cut', 'hazelnut-good', 'hazelnut-hole', 'hazelnut-print', 'leather-color', 'leather-cut', 'leather-fold', 'leather-glue', 'leather-good', 'leather-poke', 'metal_nut-bent', 'metal_nut-color', 'metal_nut-flip', 'metal_nut-good', 'metal_nut-scratch', 'pill-color', 'pill-combined', 'pill-contamination', 'pill-crack', 'pill-faulty_imprint', 'pill-good', 'pill-pill_type', 'pill-scratch', 'screw-good', 'screw-manipulated_front', 'screw-scratch_head', 'screw-scratch_neck', 'screw-thread_side', 'screw-thread_top', 'tile-crack', 'tile-glue_strip', 'tile-good', 'tile-gray_stroke', 'tile-oil', 'tile-rough', 'toothbrush-defective', 'toothbrush-good', 'transistor-bent_lead', 'transistor-cut_lead', 'transistor-damaged_case', 'transistor-good', 'transistor-misplaced', 'wood-color', 'wood-combined', 'wood-good', 'wood-hole', 'wood-liquid', 'wood-scratch', 'zipper-broken_teeth', 'zipper-combined', 'zipper-fabric_border', 'zipper-fabric_interior', 'zipper-good', 'zipper-rough', 'zipper-split_teeth', 'zipper-squeezed_teeth']
original_labels = dict(zip(range(len(labels)),labels))
sub['label'] = sub['label'].replace(original_labels)
sub

In [ ]:
sub.to_csv('./data/0512_2_submission.csv', index=False)

In [ ]:
# 정상 샘플 개수
good_cnt = 0
for i in range(len(sub)):
    if sub['label'][i][-4:] == 'good':
        good_cnt += 1
print(good_cnt)

In [ ]:
# 학습에 사용한 모델의 batch_size, epoch, img_size 기록
print('batch_size =', args.batch_size)
print('epochs =', args.epochs)
print('img_size =', args.img_size)
print('patience =', args.patience)

In [ ]:
print('model =', args.encoder_name)